In [8]:
import cv2
import numpy as np
from math import acos,pi,sqrt,exp
from matplotlib import pyplot as plt
from tkinter import *
from tkinter import filedialog
import tkinter
from PIL import ImageTk,Image

def compute_Hue(B,G,R):
    angle=0
    if B!=G!=R:
        angle=0.5*((R-G)+(R-B))/sqrt((R-G)*(R-G)+(R-B)*(G-B))
    return acos(angle) if B<=G else (2*pi - acos(angle))
#Color to HSI Convert
def convert():
    window=Tk()
    window.title('Image Viewer')
    src=cv2.imread(filedialog.askopenfilename(),cv2.IMREAD_COLOR)
    window.mainloop()
    height,width = src.shape[0],src.shape[1]

    I=np.zeros((height,width))
    S=np.zeros((height,width))
    H=np.zeros((height,width))

    for i in range(height):
        for j in range(width):
            B,G,R=src[i][j][0]/255., src[i][j][1]/255., src[i][j][2]/255.
            I[i][j]=(B+G+R)/3.
            if B+G+R !=0:
                S[i][j]=1-3*np.min([B,G,R])/(B+G+R)
            H[i][j]=compute_Hue(B,G,R)
    dst=np.zeros((height,width,3),dtype=np.uint8)
    for i in range(height):
        for j in range(width):
            if H[i][j]>=0.25 and H[i][j] <=0.6:
                dst[i][j]=src[i][j]
    cv2.imshow("src",src)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    #HSI Normalize
    I=I*255
    S=S*255
    H=H*255/(2*pi)
    I=np.asarray(I,dtype=np.uint8)
    S=np.asarray(S,dtype=np.uint8)
    H=np.asarray(H,dtype=np.uint8)
    return I
#Gaussian filter
#openCV
def Gaussian():
    I=convert()
    result1=cv2.GaussianBlur(I,(3,3),0)
    result2=cv2.GaussianBlur(I,(5,5),0)
    result3=cv2.GaussianBlur(I,(7,7),0)
    results=np.hstack((I,result1,result2,result3))
    cv2.imshow("Gaussian : Original | 3x3 | 5x5 | 7x7",results)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    result1=cv2.GaussianBlur(I,(3,3),0)
    result1_2=cv2.GaussianBlur(I,(3,3),0.4)
    result1_3=cv2.GaussianBlur(I,(3,3),1.4)
    results2=np.hstack((I,result1,result1_2,result1_3))
    cv2.imshow("Gaussian: Original | var = 0.4 | var = 1.4",results2)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
#average filter
def average():
    I=convert()
    kernel1=np.ones((3,3),np.float32)/9
    kernel2=np.ones((5,5),np.float32)/25
    kernel3=np.ones((7,7),np.float32)/49
    result1=cv2.filter2D(I,-1,kernel1)
    result2=cv2.filter2D(I,-1,kernel2)
    result3=cv2.filter2D(I,-1,kernel3)
    results=np.hstack((I,result1,result2,result3))
    cv2.imshow("Average : Original | 3x3 | 5x5 | 7x7",results)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
#median filer
def median():
    I=convert()
    result1=cv2.medianBlur(I,3)
    result2=cv2.medianBlur(I,5)
    result3=cv2.medianBlur(I,7)
    results=np.hstack((I,result1,result2,result3))
    cv2.imshow("Median : Original | 3x3 | 5x5 | 7x7",results)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
#HighBoost Filter
def highBoost():
    I=convert()
    A_value0=[[0,0,0],
           [0,1,0],
            [0,0,0]]
    A_value1=[[0,0,0],
           [0,1.2,0],
            [0,0,0]]
    A_value2=[[0,0,0],
           [0,1.5,0],
            [0,0,0]]
    kernel1=[[0,-1,0],
           [-1,4,-1],
            [0,-1,0]]
    kernel2=[[0,-1,0],
           [-1,8,-1],
            [0,-1,0]]
    A_value0=np.array(A_value0)
    A_value1=np.array(A_value1)
    A_value2=np.array(A_value2)
    kernel1=np.array(kernel1)
    kernel2=np.array(kernel2)
    high_boost11=A_value0+kernel1
    high_boost12=A_value0+kernel2
    high_boost21=A_value1+kernel1
    high_boost22=A_value1+kernel2
    high_boost31=A_value2+kernel1
    high_boost32=A_value2+kernel2
    result1=cv2.filter2D(I,-1,high_boost11)
    result2=cv2.filter2D(I,-1,high_boost12)
    result3=cv2.filter2D(I,-1,high_boost21)
    result4=cv2.filter2D(I,-1,high_boost22)
    result5=cv2.filter2D(I,-1,high_boost31)
    result6=cv2.filter2D(I,-1,high_boost32)
    results=np.hstack((I,result1,result3,result5,result2,result4,result6))
    cv2.imshow("High Boost : Original | A=1 | A=1.2 | A=1.5",results)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
#Color Space Median filter
def ColorMedian():
    window=Tk()
    window.title('Image Viewer')
    src=cv2.imread(filedialog.askopenfilename(),cv2.IMREAD_COLOR)
    window.mainloop()
    result1=cv2.medianBlur(src,3)
    result2=cv2.medianBlur(src,5)
    result3=cv2.medianBlur(src,7)
    result4=cv2.medianBlur(src,15)
    results=np.hstack((src,result1,result2,result3,result4))
    cv2.imshow("Color Space Median : Original | 3x3 | 5x5 | 7x7 | 15x15",results)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    result1=cv2.medianBlur(src,3)
#Prewitt edge detection
def prewittEdge():
    I=convert()
    prewitt_x=np.array(([[-1,0,1],[-1,0,1],[-1,0,1]]))
    prewitt_y=np.array(([[-1,-1,-1],[0,0,0],[1,1,1]]))
    result_y=cv2.filter2D(I,-1,prewitt_y)
    result_x=cv2.filter2D(I,-1,prewitt_x)
    new_result=np.array(result_y+result_x)
    results=np.hstack((I,result_y,result_x,new_result))
    cv2.imshow("Prewitt Edge Detection",results)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
#Sobel edge detection
def sobelEdge():
    I=convert()
    sobel_y=np.array([[-1,-2,-1],[0,0,0],[1,2,1]])
    sobel_x=np.array([[-1,0,1],[-2,0,2],[-1,0,1]])
    result_y=cv2.filter2D(I,-1,sobel_y)
    result_x=cv2.filter2D(I,-1,sobel_x)
    new_result=np.array(result_y+result_x)
    results=np.hstack((I,result_y,result_x,new_result))
    cv2.imshow("Sobel Edge Detection",results)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
#LoG Edge-Mask adjustment
def logEdge():
    I=convert()
    result1=cv2.GaussianBlur(I,(3,3),0)
    result2=cv2.GaussianBlur(I,(5,5),0)
    result3=cv2.GaussianBlur(I,(7,7),0)
    filter_result1=cv2.Laplacian(result1,cv2.CV_64F,ksize=3)
    filter_result2=cv2.Laplacian(result2,cv2.CV_64F,ksize=5)
    filter_result3=cv2.Laplacian(result3,cv2.CV_64F,ksize=7)
    results=np.hstack((I,filter_result1,filter_result2,filter_result3))
    cv2.imshow("LoG Edge Detection",results)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
#Canny edge operator
def cannyEdge():
    I=convert()
    result1=cv2.GaussianBlur(I,(3,3),0)
    result2=cv2.medianBlur(I,3)
    result1_1=cv2.Canny(result1,100,200)
    result2_1=cv2.Canny(result2,100,200)
    results=np.hstack((I,result1_1,result2_1))
    cv2.imshow("Canny Edge Detection",results)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
win=Tk()
win.geometry('200x150')  
win.title('Image PhotoShop')
button1 =  Button(win, text="Gaussian", command=Gaussian).pack()
button2 =  Button(win, text="Average", command=average).pack()
button3 =  Button(win, text="Median", command=median).pack()
button4 =  Button(win, text="HighBoost", command=highBoost).pack()
button5 =  Button(win, text="ColorMedian", command=ColorMedian).pack()
button6 =  Button(win, text="Prewitt Edge", command=prewittEdge).pack()
button7 =  Button(win, text="Sobel Edge", command=sobelEdge).pack()
button8 =  Button(win, text="LoG Edge", command=logEdge).pack()
button9 =  Button(win, text="Canny Edge", command=cannyEdge).pack()
win.mainloop()

